In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv('mnist_train_small.csv')
test = pd.read_csv('mnist_test.csv')

In [4]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
X_train = X_train/255
X_test = test.iloc[:,1:]
y_test = test.iloc[:,0]
X_test = X_test/255
X_train = X_train.values.reshape(-1, 28, 28, 1)  # -1 all values *unlimited*
X_test = X_test.values.reshape(-1, 28, 28, 1)
y_train_cat = to_categorical(y_train, num_classes=10) #need to have 10 nodes at output layer (binary nodes for each )

In [5]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(19999, 28, 28, 1) (9999, 28, 28, 1) (19999,) (9999,)


In [6]:
#Build Model - convolution, non-linear transformation, pooling, flatten
Model = tf.keras.Sequential()
Model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',
                                activation = 'relu',input_shape=(28,28,1)))
Model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
Model.add(tf.keras.layers.Dropout(0.2))
Model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',
                                activation = 'relu',input_shape=(28,28,1)))
Model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
Model.add(tf.keras.layers.Dropout(0.2)) 
Model.add(tf.keras.layers.Flatten())

#build neural network
Model.add(tf.keras.layers.Dense(units=256,activation='relu'))
Model.add(tf.keras.layers.Dropout(0.2))
Model.add(tf.keras.layers.Dense(units=10,activation='softmax'))

#result 1 - dropout = 0.3, kernel_size = 3,3, optimizer = rmsprop 98.36%
#result 2 - dropout = 0.3, kernel_size = 3,3, optimizer = adam 99.5%
#result 3 - dropout = 0.3, kernel_size = 5,5, optimizer = rmsprop 99.05%
#result 4 - dropout = 0.3, kernel_size = 5,5, optimizer = adam 99.14%
#result 5 - dropout = 0.2, kernel_size = 3,3, optimizer = adam 99.4%

In [7]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0

In [8]:
Model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [9]:
Model.fit(X_train,y_train,batch_size=32,epochs=30,validation_data=(X_test,y_test))

Epoch 1/30
625/625 [==============================] - 21s 32ms/step - loss: 0.6187 - accuracy: 0.8012 - val_loss: 0.1022 - val_accuracy: 0.9696
Epoch 2/30
625/625 [==============================] - 22s 36ms/step - loss: 0.1155 - accuracy: 0.9647 - val_loss: 0.0610 - val_accuracy: 0.9799
Epoch 3/30
625/625 [==============================] - 20s 32ms/step - loss: 0.0811 - accuracy: 0.9726 - val_loss: 0.0475 - val_accuracy: 0.9848
Epoch 4/30
625/625 [==============================] - 20s 32ms/step - loss: 0.0562 - accuracy: 0.9801 - val_loss: 0.0449 - val_accuracy: 0.9846
Epoch 5/30
625/625 [==============================] - 19s 30ms/step - loss: 0.0446 - accuracy: 0.9847 - val_loss: 0.0475 - val_accuracy: 0.9847
Epoch 6/30
625/625 [==============================] - 20s 32ms/step - loss: 0.0387 - accuracy: 0.9880 - val_loss: 0.0359 - val_accuracy: 0.9876
Epoch 7/30
625/625 [==============================] - 22s 35ms/step - loss: 0.0311 - accuracy: 0.9899 - val_loss: 0.0425 - val_accuracy:

In [47]:
y_pred = np.argmax(Model.predict(X_test), axis=-1)

In [48]:
accuracy_score(y_test,y_pred)

0.9892989298929893

In [41]:
# y_result = np.argmax(y_pred,axis=1)
# y_result.reshape(-1,1)
# y_result = pd.Series(y_result,name='Label')
# results = pd.Series(range(1,28001),name='ImageId')
# results_5 = pd.concat([results,y_result],axis=1)

In [42]:
# results.to_csv('digitrecog_rmsprop.csv',index=False)
# results_2.to_csv('digitrecog_adam.csv',index=False)
# results_3.to_csv('digitrecog_rmsprop2',index=False)
# results_4.to_csv('digitrecog_adam2.csv',index=False)
# results_5.to_csv('digitrecog_adam_dropout0.2.csv',index=False)